In [1]:
import pandas as pd
from pymysql import connect
from sqlalchemy import create_engine

In [15]:
df = pd.read_excel('C:/mestrado-ifpb/trabalho-infantil/remote_data/RELATORIO_DTB_BRASIL_MUNICIPIO.xls', dtype=str)

In [3]:
df.head()

,UF,Nome_UF,Região Geográfica Intermediária,Nome Região Geográfica Intermediária,Região Geográfica Imediata,Nome Região Geográfica Imediata,Mesorregião Geográfica,Nome_Mesorregião,Microrregião Geográfica,Nome_Microrregião,Município,Código Município Completo,Nome_Município
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,15,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,2,Leste Rondoniense,6,Cacoal,379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,2,Leste Rondoniense,5,Alvorada D'Oeste,346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,2,Leste Rondoniense,3,Ariquemes,23,1100023,Ariquemes


In [4]:
df.columns

Index(['UF', 'Nome_UF', 'Região Geográfica Intermediária',
       'Nome Região Geográfica Intermediária', 'Região Geográfica Imediata',
       'Nome Região Geográfica Imediata', 'Mesorregião Geográfica',
       'Nome_Mesorregião', 'Microrregião Geográfica', 'Nome_Microrregião',
       'Município', 'Código Município Completo', 'Nome_Município'],
      dtype='object')

In [5]:
change_names = {'UF':'CDUF', 
                'Nome_UF':'NOUF', 
                'Região Geográfica Intermediária':'CDRGI',
                'Nome Região Geográfica Intermediária':'NORGI', 
                'Região Geográfica Imediata':'RGInt',
                'Nome Região Geográfica Imediata':'NORGInt', 
                'Mesorregião Geográfica':'CDMesorregiao',
                'Nome_Mesorregião':'NOMesorregiao',
                'Microrregião Geográfica':'CDMicrorregiao',
                'Nome_Microrregião':'NOMicrorregiao',
                'Município':'IndexMunicipio', 
                'Código Município Completo':'CDMunicipio',
                'Nome_Município':'NOMunicipio'}

In [18]:
df = df.rename(columns=change_names)
df.head()

,CDUF,NOUF,CDRGI,NORGI,RGInt,NORGInt,CDMesorregiao,NOMesorregiao,CDMicrorregiao,NOMicrorregiao,IndexMunicipio,CDMunicipio,NOMunicipio
0,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00015,1100015,Alta Floresta D'Oeste
1,11,Rondônia,1102,Ji-Paraná,110005,Cacoal,02,Leste Rondoniense,006,Cacoal,00379,1100379,Alto Alegre dos Parecis
2,11,Rondônia,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00403,1100403,Alto Paraíso
3,11,Rondônia,1102,Ji-Paraná,110004,Ji-Paraná,02,Leste Rondoniense,005,Alvorada D'Oeste,00346,1100346,Alvorada D'Oeste
4,11,Rondônia,1101,Porto Velho,110002,Ariquemes,02,Leste Rondoniense,003,Ariquemes,00023,1100023,Ariquemes


In [3]:
def cria_conexao(database:str, passwd: str,host:str="127.0.0.1", user:str='root'):
    con = connect(host=host,
                  user=user,
              password=passwd, database=database)
    cursor = con.cursor()
    return (con, cursor)

In [7]:
ufs = {'11': 'RO', '12': 'AC', '13': 'AM', '14': 'RR', '15': 'PA', '16': 'AP', '17': 'TO',
           '21': 'MA', '22': 'PI', '23': 'CE', '24': 'RN', '25': 'PB', '26': 'PE', '27': 'AL',
           '28': 'SE', '29': 'BA', '31': 'MG', '32': 'ES', '33': 'RJ', '35': 'SP', '41': 'PR',
           '42': 'SC', '43': 'RS', '50': 'MS', '51': 'MT', '52': 'GO', '53': 'DF'}

In [20]:
df['UF'] = [ufs[uf] for uf in df['CDUF']]

In [13]:
ufs['11']

'RO'

In [19]:
df.columns

Index(['CDUF', 'NOUF', 'CDRGI', 'NORGI', 'RGInt', 'NORGInt', 'CDMesorregiao',
       'NOMesorregiao', 'CDMicrorregiao', 'NOMicrorregiao', 'IndexMunicipio',
       'CDMunicipio', 'NOMunicipio'],
      dtype='object')

In [24]:
regioes = {'Norte': ['AC', 'AP', 'AM', 'PA', 'RO', 'RR', 'TO'],
               'Nordeste': ['AL', 'BA', 'CE', 'MA', 'PE', 'PB', 'PI', 'RN', 'SE'],
               'Sudeste': ['ES', 'MG', 'RJ', 'SP'],
               'Centro-Oeste': ['DF', 'MS', 'GO', 'MT'],
               'Sul': ['PR', 'SC', 'RS']
               }
lista=[]  
for uf in df['UF']:
        #[key  if uf in regioes[key] else None for key in regioes.keys()]
    #lista = [key for key in regioes.keys() if uf in regioes[key]]
    for key in regioes.keys():
        if uf in regioes[key]:
            lista.append(key)
df['Regiao'] = lista

In [26]:
df.tail()

,CDUF,NOUF,CDRGI,NORGI,RGInt,NORGInt,CDMesorregiao,NOMesorregiao,CDMicrorregiao,NOMicrorregiao,IndexMunicipio,CDMunicipio,NOMunicipio,UF,Regiao
5565,52,Goiás,5201,Goiânia,520002,Anápolis,05,Sul Goiano,016,Pires do Rio,22005,5222005,Vianópolis,GO,Centro-Oeste
5566,52,Goiás,5202,Itumbiara,520009,Piracanjuba,05,Sul Goiano,015,Meia Ponte,22054,5222054,Vicentinópolis,GO,Centro-Oeste
5567,52,Goiás,5206,Luziânia - Águas Lindas de Goiás,520022,Flores de Goiás,04,Leste Goiano,012,Entorno de Brasília,22203,5222203,Vila Boa,GO,Centro-Oeste
5568,52,Goiás,5205,Porangatu - Uruaçu,520018,Ceres - Rialma - Goianésia,04,Leste Goiano,012,Entorno de Brasília,22302,5222302,Vila Propício,GO,Centro-Oeste
5569,53,Distrito Federal,5301,Distrito Federal,530001,Distrito Federal,01,Distrito Federal,001,Brasília,00108,5300108,Brasília,DF,Centro-Oeste
